### Método de interpolação dasimétrica

A coluna `peso_dom` representa a proporção de domicílios do setor censitário que estão dentro daquele hexágono.

Para analisar as variáveis do Censo 2022 por hexágono, o código de análise será:

$$ValorNoHexagono = ValorNoSetor \times peso\_dom$$

#### Exemplo: 

Se  o `Setor X` tem 1000 domicílios, e o `Hexágono A` (dentro do `Setor X`) tem um `peso_dom` de $0,15$, o cálculo será: $1000 \times 0.15 = 150$. Ou seja, haverá 150 domicílios estimados no Hexágono A.

In [ ]:
import pandas as pd
import os

# Caminhos
path_v1 = '../data/raw/h3_past/br_h3_res9_v1.parquet'
chunks_dir = '../data/raw/h3_past/chunks_uf_cnefe_domicilios'
output_path = '../data/raw/h3/br_h3_res9.parquet'

print("🚀 Iniciando consolidação com cálculo de peso...")

# 1. Carregar a MALHA BASE primeiro (ela contém o vínculo H3 <-> SETOR)
cols_base = ['h3_id', 'cd_setor', 'cd_mun', 'nm_mun', 'cd_uf', 'nm_uf']
df_base = pd.read_parquet(path_v1, columns=cols_base)
print(f"✅ Malha base carregada: {len(df_base)} hexágonos.")

# 2. Carregar e consolidar os DOMICÍLIOS (CNEFE)
list_domicilios = []
for file_name in os.listdir(chunks_dir):
    if file_name.endswith('.parquet'):
        temp_df = pd.read_parquet(os.path.join(chunks_dir, file_name))
        # Renomeia para o padrão curto que você pediu
        temp_df = temp_df.rename(columns={'qtd_domicilios': 'qtd_dom'})
        list_domicilios.append(temp_df)

df_dom = pd.concat(list_domicilios, ignore_index=True)
print(f"✅ Domicílios carregados: {len(df_dom)} registros.")

# 3. CRUZAMENTO (Merge) - Unir a contagem de domicílios à malha que tem os setores
# Usamos inner para já manter apenas o que tem correspondência e domicílios
df_final = pd.merge(df_base, df_dom, on='h3_id', how='inner')

# 4. AGORA SIM: Calcular o peso_dom (Lógica Dasimétrica)
# Agora o df_final tem 'cd_setor' e 'qtd_dom' juntos!
print("⚖️ Calculando pesos dasimétricos...")

# Soma de domicílios por setor
df_final['total_dom_setor'] = df_final.groupby('cd_setor')['qtd_dom'].transform('sum')

# Peso = fração do setor dentro do hexágono
df_final['peso_dom'] = df_final['qtd_dom'] / df_final['total_dom_setor']

# 5. Limpeza Final
# Se o total_dom_setor for 0 (evitar divisão por zero), preenchemos peso com 0
df_final['peso_dom'] = df_final['peso_dom'].fillna(0)
df_final = df_final.drop(columns=['total_dom_setor'])

# Remover hexágonos com 0 domicílios (opcional, conforme sua regra)
df_final = df_final[df_final['qtd_dom'] > 0]

# 6. Salvar
df_final.to_parquet(output_path, index=False, compression='snappy')

print(f"\n✨ Sucesso! Arquivo gerado: {output_path}")
print(f"📊 Colunas: {df_final.columns.tolist()}")
print(f"📍 Total de hexágonos habitados: {len(df_final)}")

🚀 Iniciando consolidação com cálculo de peso...
✅ Malha base carregada: 4572203 hexágonos.
✅ Domicílios carregados: 4576063 registros.
⚖️ Calculando pesos dasimétricos...

✨ Sucesso! Arquivo gerado: ../data/raw/h3/br_h3_res9.parquet
📊 Colunas: ['h3_id', 'cd_setor', 'cd_mun', 'nm_mun', 'cd_uf', 'nm_uf', 'qtd_dom', 'peso_dom']
📍 Total de hexágonos habitados: 4573444
